In [1]:
!git clone https://github.com/Nneji123/DeOldify.git

Cloning into 'DeOldify'...
remote: Enumerating objects: 2404, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 2404 (delta 18), reused 45 (delta 13), pack-reused 2344
Receiving objects: 100% (2404/2404), 83.19 MiB | 34.17 MiB/s, done.
Resolving deltas: 100% (1099/1099), done.


In [2]:
!cd DeOldify/

In [3]:
import os
os.chdir("/content/DeOldify/")
!pip install -r colab_requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 214 kB 5.0 MB/s 
     |████████████████████████████████| 129 kB 53.6 MB/s 
     |████████████████████████████████| 1.9 MB 65.5 MB/s 
     |████████████████████████████████| 2.9 MB 51.0 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
     |████████████████████████████████| 79 kB 10.7 MB/s 
     |████████████████████████████████| 45 kB 4.2 MB/s 
     |████████████████████████████████| 78 kB 8.6 MB/s 
     |████████████████████████████████| 355 kB 70.5 MB/s 
     |████████████████████████████████| 86.3 MB 39 kB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 3.8 MB 47.6 MB/s 
     |████████████████████████████████| 449 kB 66.6 MB/s 
     |████████████████████████████████| 50 kB 7.3 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 80 kB 

In [4]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!ngrok authtoken 29vr8YhWZ9CDHrUq2kr0CpUA0e8_6ik4hU5GjJZeAYagTH5i4

--2022-07-18 11:16:01--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7189521 (6.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.1’

ngrok-v3-stable-lin 100%[===================>]   6.86M  27.0MB/s    in 0.3s    

2022-07-18 11:16:02 (27.0 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.1’ saved [7189521/7189521]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!mkdir -p /root/.torch/models
!mkdir -p /content/DeOldify/models

!wget -O /root/.torch/models/vgg16_bn-6c64b313.pth https://download.pytorch.org/models/vgg16_bn-6c64b313.pth

!wget -O /root/.torch/models/resnet34-333f7ec4.pth https://download.pytorch.org/models/resnet34-333f7ec4.pth

!wget -O /content/DeOldify/models/ColorizeArtistic_gen.pth https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth

--2022-07-18 11:16:05--  https://download.pytorch.org/models/vgg16_bn-6c64b313.pth
Resolving download.pytorch.org (download.pytorch.org)... 18.160.200.126, 18.160.200.77, 18.160.200.112, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.160.200.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553507836 (528M) [application/octet-stream]
Saving to: ‘/root/.torch/models/vgg16_bn-6c64b313.pth’

/root/.torch/models 100%[===================>] 527.87M   241MB/s    in 2.2s    

2022-07-18 11:16:08 (241 MB/s) - ‘/root/.torch/models/vgg16_bn-6c64b313.pth’ saved [553507836/553507836]

--2022-07-18 11:16:08--  https://download.pytorch.org/models/resnet34-333f7ec4.pth
Resolving download.pytorch.org (download.pytorch.org)... 18.160.200.126, 18.160.200.77, 18.160.200.112, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.160.200.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87306240 (83M) [application/octet-str

In [6]:
import os
os.chdir("/content/DeOldify/")
import uvicorn
from fastapi import FastAPI, File, UploadFile, Response
from fastapi.responses import StreamingResponse, FileResponse
import numpy as np
import io
from PIL import Image
import cv2
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)
import torch
import fastai
from ISR.models import RDN
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

app = FastAPI()

rdn = RDN(weights='noise-cancel')

colorizer = get_image_colorizer(artistic=True)


@app.get('/')
def home():
    return {'Title': 'Super Resolution and Colorisation API'}


# Endpoint for enhancing resolution and colorization of image
@app.post("/enhance_and_colorise")
async def root(file: UploadFile = File(...)):

    
    contents = io.BytesIO(await file.read())
    
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR) 
    
    cv2.imwrite("new.jpg",img)
    
    sr_img = rdn.predict(img, by_patch_of_size=300)
    
    res, im_png = cv2.imencode(".png", sr_img)
    
    images = io.BytesIO(im_png.tobytes())

    sr2_img = colorizer.get_transformed_image(images, render_factor=35)
    
    new_img = np.array(sr2_img)
    # img = cv2.imdecode(np.array(sr_img), cv2.IMREAD_COLOR)
    
    res, im2_png = cv2.imencode(".png", new_img) 

    return StreamingResponse(io.BytesIO(im2_png.tobytes()), media_type="image/png")


# endpoint for just enhancing the image
@app.post("/enhance")
async def root(file: UploadFile = File(...)):

    # image = load_image_into_numpy_array(await file.read())

    contents = io.BytesIO(await file.read())
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    sr_img = rdn.predict(img, by_patch_of_size=300)

    res, im_png = cv2.imencode(".png", sr_img)

    return StreamingResponse(io.BytesIO(im_png.tobytes()), media_type="image/png")

# endpoint for just colorising the image
# Please review this section I may have mixed up something
@app.post("/colorise")
async def root(file: UploadFile = File(...)):

    
    contents = io.BytesIO(await file.read())
    
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR) 
    
    cv2.imwrite("new.jpg",img)
    
    #sr_img = rdn.predict(img, by_patch_of_size=300)
    
   # res, im_png = cv2.imencode(".png", sr_img)
    
    #images = io.BytesIO(im_png.tobytes())

    sr2_img = colorizer.get_transformed_image("new.jpg", render_factor=35)
    
    new_img = np.array(sr2_img)
    # img = cv2.imdecode(np.array(sr_img), cv2.IMREAD_COLOR)
    
    res, im2_png = cv2.imencode(".png", new_img) 

    return StreamingResponse(io.BytesIO(im2_png.tobytes()), media_type="image/png")







66076672/66071288 [==============================] - 10s 0us/step


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:136: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavio

  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [7]:
from colabcode import ColabCode
cc = ColabCode(port=18000, code=False)
cc.run_app(app=app)

Public URL: NgrokTunnel: "http://0ce8-107-178-212-79.ngrok.io" -> "http://localhost:18000"


INFO:     Started server process [70]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:18000 (Press CTRL+C to quit)


INFO:     102.89.41.253:0 - "GET / HTTP/1.1" 200 OK
INFO:     102.89.41.253:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.89.41.253:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.89.41.253:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [70]
